In [ ]:
%matplotlib inline

In [ ]:
import gc
import pathlib
import sys

In [ ]:
import joblib
import lightgbm as lgb
import pandas as pd

In [ ]:
root_dir = "../.."

In [ ]:
root_dir_path = pathlib.Path(root_dir)
data_dir_path = root_dir_path / "data"
processed_dir_path = data_dir_path / "processed"
train_path = processed_dir_path / "train.parquet"
models_dir_path = root_dir_path / "models"
model_path = models_dir_path / "model.joblib"
src_dir_path = root_dir_path / "src"

In [ ]:
sys.path.append(str(src_dir_path))

In [ ]:
from package.constants import *

In [ ]:
train = pd.read_parquet(train_path)

In [ ]:
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/135896
is_train = (train["date"] >= "2015-03-28") & (train["date"] < "2016-03-28")
is_valid = train["date"] >= "2016-03-28"

In [ ]:
X = train.loc[is_train, features]
X_valid = train.loc[is_valid, features]
y = train.loc[is_train, "demand"]
y_valid = train.loc[is_valid, "demand"]
sample_weight = train.loc[is_train, "sample_weight"]
sample_weight_valid = train.loc[is_valid, "sample_weight"]

In [ ]:
del train
del is_train
del is_valid

In [ ]:
gc.collect()

In [ ]:
X.info()

In [ ]:
X_valid.info()

In [ ]:
model = lgb.LGBMRegressor(
    importance_type="gain", learning_rate=0.05, n_estimators=1_000, random_state=0
)

In [ ]:
%%time
model.fit(
    X,
    y,
    sample_weight=sample_weight,
    eval_set=[(X, y), (X_valid, y_valid)],
    eval_sample_weight=[sample_weight_valid],
    eval_metric="rmse",
    early_stopping_rounds=100,
    categorical_feature=categorical_features,
)

In [ ]:
ax = lgb.plot_metric(model, metric="rmse", figsize=(16, 9))

In [ ]:
ax = lgb.plot_importance(model, figsize=(16, 9))

In [ ]:
joblib.dump(model, model_path)